
### **Prep**


In [1]:
#the module 'sys' allows istalling module from inside Jupyter
import sys


import numpy as np

import pandas as pd

#Natrual Language ToolKit (NLTK)

import nltk

from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /Users/xun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/xun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/xun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/xun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

Below we define two functions that will display the results of fitting a topic model, to be used later:

*Note: these functions are not the focus of the lab, therefore we'll not be discussing them, but you are welcome to explore and dig into them later if you prefer.*

In [2]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

Let's **normalize** our toy_corpus and call the normalized corpus **normalized_toy_corpus**:

Since for topic modeling we need text data in the **Bag-of-Words** representation, let's **vectorize** our normalized_toy_corpus and call it **bow_toy_corpus**:

In [5]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer()

#vectorize the normalized data:
bow_toy_corpus = bow_vectorizer.fit_transform(normalized_toy_corpus)

Have a look at the Bag-of-Words representation of our corpus: **It never hurts to know how you data look like :)** Note absence of stopwords and other differences with the raw data:

#### **Topic Model (via Latent Dirichlet Allocation) on Toy Data**
Now let's **model topics** in our toy data. Given that the toy corpus is so small, we know all "topics" it contains (**what are they?**) and it will be easy for us:<br> 1) to check if the topic model results make sense; <br>2) see all the results that the topic model produces.  <br><br>
We will be using the **LatentDirichletAllocation** function which we already imported earlier (see Session Prep). The function has the following **parameters** to be set:
1. Number of topics to model: **n_components**
2. Parameter vector for the Dirichlet distribution for *topics*: **doc_topic_prior**
3. Parameter vector for the Dirichlet distribution for *words* in a topic: **topic_word_prior**

Notes on **parameter vectors for the Dirichlet distributions**: <br>
1. Although the Dirichlet distribution parameters are represented by a **vector**, for simplicity we provide one number for each parameter vector. For example, if we set the number of topics to 2 (n_components=2), the parameter vector for the Dirichlet distribution for *topics* should be a two-dimensional vector. We set doc_topic_prior=0.5 and the LatentDirichletAllocation function internally creates a two-dimensional vector (0.5,0.5). Similar logic applies to the parameter vector for the Dirichlet distribution for *words* (the dimensionality of that parameter vector is equal to the number of terms in the text corpus, which is typically very large).<br><br>
2. Remember, that we need **sparsity** in the distribution of topics across documents (i.e., some documents have a zero probability of containing some of the topics) and *sparsity* in the distribution of words in topics (i.e., some words have zero probability to be present in some topics). To induce sparsity, we need to set **doc_topic_prior** and **topic_word_prior** between 0 and 1.

Now, let's set the parameters and estimate the topic model:

Display results by showing 15 **most frequent (top)** words for each topic (we use **function display_topics** defined in Session Prep):

Note that topics do not have names or labels. **Topics are just collections of words**, following the definition of a topic in text mining. <br><br> To be precise, topics are **word vectors**, where each vector element is the **weight** (relative frequency) of the word in a topic. Let's have a look at those "word vectors". Can you see below that each word vector (topic) is a **simplex**?

### **Topic Modeling on Real Data**

The dataset here is the one we used for doing Text Classification in Lab 3. The newspaper blog posts have 4 topics: **atheism, religion, computer graphics, and space science**. Of course, we will *not use* class label information for topic modeling.

Download the data and set up the data (**news_corpus**):

In [2]:
data=pd.read_csv('universal_studio_branches.csv')
print(data.head(10))

          reviewer  rating  written_date  \
0          Kelly B     2.0  May 30, 2021   
1              Jon     1.0  May 30, 2021   
2          Nerdy P     2.0  May 30, 2021   
3        ran101278     4.0  May 29, 2021   
4  tammies20132015     5.0  May 28, 2021   
5             John     1.0  May 28, 2021   
6     annapN7702ZW     2.0  May 27, 2021   
7            Deb P     2.0  May 27, 2021   
8          Chuck N     1.0  May 27, 2021   
9              Jen     4.0  May 26, 2021   

                                               title  \
0  Universal is a complete Disaster - stick with ...   
1                               Food is hard to get.   
2                                       Disappointed   
3                                         My opinion   
4                  The Bourne Stuntacular...MUST SEE   
5                             This is not a vacation   
6                                      Expected More   
7                                  Disapointing.....   
8        Gr

In [5]:
text=data.review_text.values
type(text)

numpy.ndarray

In [6]:
#normalize data
normalized_corpus_news = normalize_corpus(text)

#define a Bag-of-Words vecgtorizer
bow_vectorizer_news = CountVectorizer(max_features=1000)

#vectorize data
bow_news_corpus = bow_vectorizer_news.fit_transform(normalized_corpus_news)

In [7]:
lda_news = LatentDirichletAllocation(n_components=5, max_iter=500,
                                     doc_topic_prior = 0.25,
                                     topic_word_prior = 0.25).fit(bow_news_corpus)

In [11]:
word_weights = lda_news.components_ / lda_news.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_vectorizer_news.get_feature_names(), 
                               columns = ["Topic_" + str(i) for i in range(5)])

word_weights_df.sort_values(by='Topic_0',ascending=False).head(15)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
park,0.021220,1.044953e-02,3.340122e-02,2.596376e-02,4.505873e-02
people,0.014238,1.165629e-03,5.927128e-07,4.383927e-03,5.709567e-07
universal,0.013987,9.320728e-03,1.453263e-02,2.697777e-03,3.849281e-02
us,0.013453,6.812128e-03,5.923136e-07,4.835988e-03,1.686476e-03
take,0.010932,1.055158e-02,2.950493e-03,5.932997e-03,2.268689e-03
line,0.010359,7.447773e-07,1.398116e-03,2.662785e-02,5.607270e-07
time,0.009894,8.035115e-03,6.996664e-03,3.238395e-02,2.108675e-02
like,0.009269,1.037181e-02,1.284914e-02,9.586760e-05,5.725668e-03
walk,0.008997,9.745701e-04,3.565183e-03,9.837586e-04,3.532188e-03
locker,0.008504,7.718435e-07,5.790872e-07,4.567965e-07,5.430756e-07


In [12]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda_news, bow_news_corpus, bow_vectorizer_news, mds='tsne')


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153  -72.269554       1        1  25.035673
0       51.163364 -159.968018       2        1  20.688525
4      -73.144554   21.523815       3        1  19.927865
2      -66.410133 -110.606438       4        1  19.330237
1       54.417324   12.443444       5        1  15.017700, topic_info=        Term          Freq         Total Category  logprob  loglift
651   potter  20281.000000  20281.000000  Default  30.0000  30.0000
386    harry  20172.000000  20172.000000  Default  29.0000  29.0000
616      pas  18915.000000  18915.000000  Default  28.0000  28.0000
715     ride  99402.000000  99402.000000  Default  27.0000  27.0000
295  express  19443.000000  19443.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
610     park   3683.792141  65482.769439   Topic5  -4.5612  -0.9819
890     time   2832.633409  39708.607482   Topic5  -4.8239  -0.7444
201      day   2731.569745  39610.334573   Topic5  -4.8603  -0.7783
674    queue   2503.027450  14761.490209   Topic5  -4.9476   0.1214
927       us   2401.491840  12566.440885   Topic5  -4.9891   0.2410

[347 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3         4  0.631905     3d
3         5  0.368053     3d
5         4  0.312352     4d
5         5  0.687234     4d
7         1  0.916035    9am
...     ...       ...    ...
996       3  0.672758  young
996       5  0.250533  young
998       3  0.784251     yr
998       4  0.076361     yr
998       5  0.138276     yr

[626 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 3, 2])

In [13]:
lda_news_topic_weights = lda_news.transform(bow_news_corpus)

In [14]:
#array of document "names" and topic "names" ("names" are just indecies)
doc_names = ["Doc_" + str(i) for i in range(len(normalized_corpus_news))]
topic_names = ["Topic_" + str(i) for i in range(5)]

#convert to dataframe
df_document_topic = pd.DataFrame(np.round(lda_news_topic_weights, 5), columns=topic_names, index=doc_names)
df_document_topic.head(15)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
Doc_0,0.51723,0.00247,0.02920,0.43227,0.01883
Doc_1,0.52954,0.01627,0.01566,0.29845,0.14008
Doc_2,0.75160,0.00332,0.00345,0.17388,0.06775
Doc_3,0.57943,0.01716,0.01608,0.37172,0.01561
Doc_4,0.22183,0.14848,0.00753,0.20130,0.42085
Doc_5,0.19180,0.01456,0.01451,0.16501,0.61412
Doc_6,0.66917,0.00859,0.00848,0.17895,0.13482
Doc_7,0.52788,0.00320,0.05528,0.24934,0.16430
Doc_8,0.59171,0.00284,0.10940,0.29313,0.00292
Doc_9,0.40790,0.00073,0.17579,0.32029,0.09528


In [15]:
#vector of indecies for columns with the highest value by each row in df_document_topic
dominant_topic = np.argmax(df_document_topic.values, axis=1)

#add dominant_topic as a column to df_document_topic
df_document_topic['dominant_topic'] = dominant_topic
df_document_topic.head(15)

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,dominant_topic
Doc_0,0.51723,0.00247,0.02920,0.43227,0.01883,0
Doc_1,0.52954,0.01627,0.01566,0.29845,0.14008,0
Doc_2,0.75160,0.00332,0.00345,0.17388,0.06775,0
Doc_3,0.57943,0.01716,0.01608,0.37172,0.01561,0
Doc_4,0.22183,0.14848,0.00753,0.20130,0.42085,4
Doc_5,0.19180,0.01456,0.01451,0.16501,0.61412,4
Doc_6,0.66917,0.00859,0.00848,0.17895,0.13482,0
Doc_7,0.52788,0.00320,0.05528,0.24934,0.16430,0
Doc_8,0.59171,0.00284,0.10940,0.29313,0.00292,0
Doc_9,0.40790,0.00073,0.17579,0.32029,0.09528,0


Now, let's normalize the corpus and create the Bag-of-Words representation of the data. We'll limit the number of features to **1000 most frequent features** to compute the topic model faster. 

Now let's fit the topic model. We need to **set the number of topics** first. We are *lucky to know* that there are **4 topics** (atheism, religion, computer graphics, and space science) and it will allow us to judge the performance of the topic model better.

**Note**: It will take a couple of minutes for the estimation to finish. The larger the number of iterations (max_iter) you allow for, the longer it takes. You can also find that you might get a slightly different result (for the estimated weights) if you rerun the model (qualitatively, the result is likely to stay the same). This behavior is due to the stochastic nature of the model. Estimation starts from a random guess for the weights and iteratively improves. When we do not allow for a large number of iterations, we are more likely to be affected by the random starting values. 

Display results with top 10 words for each topic:

Display **word vectors** (words are in alphabetical order) for each topic. Each column is a topic:

Now, **sort by word weights in Topic 0** (descending order) and see the weights by 10 most frequent words in Topic 0:

### **Topic Model Visualization**

You can **visualize** the topics: topic size, frequency of words in a topic and so on.

In this visualization, you can rank words in a topic by **relevancy**: do you want rare and exclusive terms (i.e. found mostly in that topic) or terms that are used frequently in that topic, but not always exclusive to that topic? Relevancy parameter is λ (0 ≤ λ ≤ 1). You can adjust it:

* small λ highlights potentially rare, but exclusive terms for the selected topic;
* large values of λ (near 1) highlight frequent, but not necessarily exclusive, terms for the selected topic;

Relevancy is measured as: 

    Relevancy = λ log[p(term | topic)] + (1 - λ) log[p(term | topic)/p(term)], 
   
   where p(term | topic) stands for word weight in a topic and p(term) stands for word's weight in a corpus.

Additional information on how to use this visualization:
* http://www.kennyshirley.com/LDAvis/
* https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf

We installed all the **pyLDAvis** module required for this visualization in Session Prep. Now let's use it:

### **How To Find Dominant Topic in a Document**

Each document contains several topics. One of the topics is **dominant**, i.e. it is the largest topic in the document. That topic gives you an answer to the question: **What is this document about?** In other words, the document's dominant topic **summarizes** the document. 

Let's assign a dominant topic to **each document** in our corpus. Weights in a word vector for a topic provide a measure of association for the word with the topic. If you sum weights for a particular topic across all words in a document, you'll get the weight of that topic in the document.

The attribute **.transform** to our function **lda_news** computes the weights of each topic in documents: 

In [22]:
lda_news_topic_weights = lda_news.transform(bow_news_corpus)

Let's convert lda_news_topic_weights into a nice-looking dataframe and have a look at the computed topic weights in documents:

The topic with the highest weight in each document is a **dominant topic**. The weights across the 4 topics sum up to 1. Let's add a column that shows dominant topic for each document:

### **Topic Model Evaluation: Log-likelihood, Perplexity and Coherence Scores**

Log-likelihood, Perplexity and Coherence Score are **measures of performance** for a topic model. They are used for comparing and discriminating between topic models estimated on the same data. Log-likelihood, perplexity and coherence scores **do not have** a baseline or a threshold values and therefore are useful only for comparing models. 

How do you specify different models? You can set **different number of topics** and also play with the **parameters of the Dirichlet distributions**. 

#### **Coherence Score**

We will use a function **CoherenceModel()** from the **gensim** module (you can also explore that package as it can be used to estimate an LDA model). The sklearn module does not have the functionality to compute the coherence score. Let's install the gensim package and the functions needed:

In [25]:
!{sys.executable} -m pip install gensim
import gensim

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

The function CoherenceModel() needs as **inputs**:

**1. Dictionary of the corpus**<br>
**2. Corpus with each document represented as Bag-of-Words**<br>
**3. An array of top words for each topic: we'll have top 20 words for each topic** 
  
We will now create those objects:

In [26]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(normalized_corpus_news[doc_id]) for doc_id in range(len(normalized_corpus_news))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news, n_words=20)

Now let's compute **the coherence score for the model overall**. We use one of the coherence metrics "u-mass" which measures semantic similarity of words in a topic, but there are other metrics as well.

*Note: You can check out different coherence metrics here if you are interested: https://dl.acm.org/doi/abs/10.1145/2684822.2685324*

In [27]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

Coherence score for the model:  -1.5086


You can also see **coherence scores by topic**:

In [28]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),4))

Coherence score by topic (higher values are better):  [-1.5913 -1.6719 -1.3431 -1.4282]


#### **Log-Likelihood Score**

To compute the log-likelihood score we use the **.score** attribute of our defined and fitted LDA function:

In [29]:
print("Log-Likelihood (higher values are better): ", lda_news.score(bow_news_corpus))

Log-Likelihood (higher values are better):  -741444.8353556208


#### **Perplexity Score**

To compute the Perplexity score we use the **.perplexity** attribute of our defined and fitted LDA function:

In [30]:
print("Perplexity (lower values are better): ", lda_news.perplexity(bow_news_corpus))

Perplexity (lower values are better):  575.2087781096856


The coherence scores by topic are [-1.5466 -1.3606 -1.4048 -1.7395]. We got coherent topics as judged by reading through the top words of each topic: we see that words are related to each other and we can use them to tell a story easily, without thinking too hard of how to include each of the top words into a story. Topic with coherence score of -1.3606 (the highest number) is of the highest quality (most coherent and likely easier for a human to inperprete). Having one topic with a drastically different coherence score might indicate that we can probably do better with a smaller number of topics. 

**Answer 3.2:**

<font color=blue>Code (complete the lines):<font>

In [34]:
#Log-Likelihood
print("Log-Likelihood (higher values are better): ", lda_news_3_topics.score(bow_news_corpus))

#Perplexity score:
print("Perplexity (lower values are better): ", lda_news_3_topics.perplexity(bow_news_corpus))

#coherence score for 3 topics:
topic_topwords_3_topics = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news_3_topics, n_words=20)
cm_3_topics = CoherenceModel(topics=topic_topwords_3_topics, 
                             corpus = news_corpus_bow, 
                             dictionary = news_dictionary, coherence='u_mass')
#overall coherence score for the model:
print("Coherence score for the model: (higher values are better)", np.round(cm_3_topics.get_coherence(), 3)) 


Log-Likelihood (higher values are better):  -743489.8926210115
Perplexity (lower values are better):  585.3797180350961
Coherence score for the model: (higher values are better) -1.44


<font color=blue>Discussion:<br><br><font>
    When we compare a model with 3 topics to the model with 4 topics using the stats (output above), we get an inconsistent picture: <br><font>
    
<font color=blue> - log-likelihood is lower for the model with 3 topics, which means 4 topics is better (3 topics: -743309.96 < 4 topics: -741363.0) <br><font>
<font color=blue>- perplexity score is higher for 3 topics, which means 4 topics is better (3 topics: 584.47 > 4 topics:  574.8)<br><font>
<font color=blue>- coherence score is higher for the model with 3 topics, which means 3 topics is better (3 topics: -1.436 > 4 topics: -1.5129)<br><font>
    
<font color=blue>Thus, coherence score is in favor of a 3-topic model, while the other two stats are in favor of a 4-topic model. 
    
<font color=blue>From experimental research, it is known that coherence score typically reflects human judgment better, so we might want to rely on coherence score more than on the other two stats. Given that in the 3-topics model, atheism and religion were combined (atheism is often viewed as a discussion of religion) we, as humans, might agree with the coherence score stat here. <font>
    
<font color=blue>You may have noticed that in the LDA model with 4 topics, the topic with top words "think", "like", "know", "people" and so on (the "atheism" topic) was a bit harder to interpret than other 3 topics. The LDA model allocated to this topic the top words that can be interpreted as related to doubt and knowledge which are important ideas in the atheism discourse. For this reason, some humans may prefer a model with 4 topics (vs. 3 topics) as the atheism topic is subtly different from the overall discussion of religion. <font>

<font color=blue>**NOTE:** that estimation results can be slightly different for different runs of the models. The output can be sensitive to the starting values of the LDA algorithm. The reason is that in the lab script we set the number of iteration (max_iter) to a relatively small number to speed up the estimation.<font>

**Answer 3.3:**

Code:

In [35]:
  #fit LDA with 2 topics:
lda_news_2_topics = LatentDirichletAllocation(n_components=2, max_iter=100,
                                              doc_topic_prior = 0.25,
                                              topic_word_prior = 0.25).fit(bow_news_corpus)

#Log-Likelihood
print("Log-Likelihood (higher values are better): ", lda_news_2_topics.score(bow_news_corpus))

#Perplexity score:
print("Perplexity (lower values are better): ", lda_news_2_topics.perplexity(bow_news_corpus))

#coherence score for 3 topics:
topic_topwords_2_topics = get_topic_words(vectorizer = bow_vectorizer_news, lda_model = lda_news_2_topics, n_words=20)
cm_2_topics = CoherenceModel(topics=topic_topwords_2_topics, 
                             corpus = news_corpus_bow, 
                             dictionary = news_dictionary, coherence='u_mass')
#overall coherence score for the model:
print("Coherence score for the model: (higher values are better)", np.round(cm_2_topics.get_coherence(), 3))  

Log-Likelihood (higher values are better):  -751871.0460985524
Perplexity (lower values are better):  628.9762131689758
Coherence score for the model: (higher values are better) -1.421


<font color=blue>Discussion:<font>

All stats consistently show that the model with 2 topics is worse than the model with 3 topics or 4 topics. In the model with 2 topics, the atheism and religion classes and the space science and computer graphics classes were combined, respectively. Many people would agree that space science and computer graphics (although both technical topics) should not be combined. Therefore, a model with 2 topics is definitely worse than a model with 3 or 4 topics. The stats and human judgment agree here.

<br>**NOTE:** Generally, you can write a simple script that selects the best topic model **automatically** based on a criterion for "best model" (log-likelihood, perplexity, or coherence score). The script can vary both parameters of the Dirichlet distributions and the number of topics, or just the number of topics.